# Medium significance 

In [1]:
%run 01-model.ipynb

'' is not a valid SBML 'SId'.


### Standard medium content of the model


In [2]:
model.medium

{'BOUNDARY_GLCe': 10.0,
 'BOUNDARY_H2Oe': 1000.0,
 'BOUNDARY_HNO3e': 1000.0,
 'BOUNDARY_O2e': 1000.0,
 'BOUNDARY_PIe': 1000.0,
 'BOUNDARY_SLFe': 1000.0,
 'BOUNDARY_Ke': 1000.0,
 'BOUNDARY_CAe': 1000.0}

#### Productivities using standard medium:

In [3]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['DRAIN_Biomass'], '/h')
    
    model.objective = model.reactions.psiM
    psi_production = model.optimize().objective_value
    print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9398547241492622 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 3.0787126793330604 mmol/gDW*h


### Changing media to an YEPD-mimicked media

In [12]:
YEPDmedium = model.medium
YEPDmedium['BOUNDARY_GLCe'] = 20
YEPDmedium['BOUNDARY_ALAe'] = 10
YEPDmedium['BOUNDARY_ARGe'] = 10
YEPDmedium['BOUNDARY_ASPe'] = 10
YEPDmedium['BOUNDARY_CYSe'] = 10
YEPDmedium['BOUNDARY_GLUe'] = 10
YEPDmedium['BOUNDARY_GLYe'] = 10
YEPDmedium['BOUNDARY_HISe'] = 10
YEPDmedium['BOUNDARY_ILEe'] = 10
YEPDmedium['BOUNDARY_LEUe'] = 10
YEPDmedium['BOUNDARY_LYSe'] = 10
YEPDmedium['BOUNDARY_METe'] = 10
YEPDmedium['BOUNDARY_PHEe'] = 10
YEPDmedium['BOUNDARY_PROe'] = 10
YEPDmedium['BOUNDARY_SERe'] = 10
YEPDmedium['BOUNDARY_THRe'] = 10
YEPDmedium['BOUNDARY_TRPe'] = 10
YEPDmedium['BOUNDARY_TYRe'] = 10
YEPDmedium['BOUNDARY_VALe'] = 10
YEPDmedium['BOUNDARY_ZNe'] = 10
YEPDmedium['BOUNDARY_FE2e'] = 10

#### Productivities using YEPD medium:

In [13]:
model.medium = YEPDmedium
solution = model.optimize()
print('Maximum theoretical biomass productivity:', solution.fluxes['DRAIN_Biomass'], '/h') 
    
model.objective = model.reactions.psiM
psi_production = model.optimize().objective_value
print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 15.115388091646892 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 53.867217378002145 mmol/gDW*h


### Changing glucose to a different carbon source
#### Productivities using the alternative carbon source:

In [4]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_SUCCe'] = 10
    model.medium = medium
    
    solution2 = model.optimize()
    print('Maximum theoretical biomass productivity:', solution2.fluxes['DRAIN_Biomass'], '/h')
    
    model.objective = model.reactions.psiM
    psi_production2 = model.optimize().objective_value
    print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production2, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.5093838996124204 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 1.5789473684210469 mmol/gDW*h


In [5]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_FRUe'] = 10
    model.medium = medium
    solution3 = model.optimize()
    print('Maximum theoretical biomass productivity:', solution3.fluxes['DRAIN_Biomass'], '/h')
    
    model.medium = medium
    model.objective = model.reactions.psiM
    psi_production3 = model.optimize().objective_value
    print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production3, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9398547241491949 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 3.0787126793330293 mmol/gDW*h


In [6]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_FUMe'] = 10
    model.medium = medium
    solution4 = model.optimize()
    print('Maximum theoretical biomass productivity:', solution4.fluxes['DRAIN_Biomass'], '/h')
  
    model.medium = medium
    model.objective = model.reactions.psiM
    psi_production4 = model.optimize().objective_value
    print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production4, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.469927362074562 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 1.5393563396664942 mmol/gDW*h


In [7]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_GLNe'] = 10
    model.medium = medium
    solution5 = model.optimize()
    print('Maximum theoretical biomass productivity:', solution5.fluxes['DRAIN_Biomass'], '/h')
    
    model.medium = medium
    model.objective = model.reactions.psiM
    psi_production5 = model.optimize().objective_value
    print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production5, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.7825699540236571 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 2.7329050022946415 mmol/gDW*h


In [8]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_GLUe'] = 10
    model.medium = medium
    solution6 = model.optimize()
    print('Maximum theoretical biomass productivity:', solution6.fluxes['DRAIN_Biomass'], '/h')
    
    model.medium = medium
    model.objective = model.reactions.psiM
    psi_production6 = model.optimize().objective_value
    print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production6, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.7825699540236892 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 2.7329050022946273 mmol/gDW*h


In [9]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_LACe'] = 10
    model.medium = medium
    solution7 = model.optimize()
    print('Maximum theoretical biomass productivity:', solution7.fluxes['DRAIN_Biomass'], '/h') 
    
    model.medium = medium
    model.objective = model.reactions.psiM
    psi_production7 = model.optimize().objective_value
    print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production7, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.4507909413688001 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 1.476670544138563 mmol/gDW*h


In [10]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_MALe'] = 10
    model.medium = medium
    solution8 = model.optimize()
    print('Maximum theoretical biomass productivity:', solution8.fluxes['DRAIN_Biomass'], '/h')
    
    model.medium = medium
    model.objective = model.reactions.psiM
    psi_production8 = model.optimize().objective_value
    print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production8, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.4699273620746312 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 1.5393563396665346 mmol/gDW*h


In [11]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_PYRe'] = 10
    model.medium = medium
    solution9 = model.optimize()
    print('Maximum theoretical biomass productivity:', solution9.fluxes['DRAIN_Biomass'], '/h')
    
    model.medium = medium
    model.objective = model.reactions.psiM
    psi_production9 = model.optimize().objective_value
    print('Maximum theoretical productivity of psilocybin_synthase reaction:', psi_production9, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.3916061350622011 /h
Maximum theoretical productivity of psilocybin_synthase reaction: 1.2827969497221146 mmol/gDW*h
